In [1]:
import os # path to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings # warning filter
import scipy as sp # pivot engineering

In [2]:
#ML model
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Default theme and settings
pd.options.display.max_columns

20

In [4]:
#warning hadle
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [11]:
#list all files under the input directory
for dirname, _, filenames in os.walk('/Users/juanp/Downloads/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/juanp/Downloads/archive\anime.csv
/Users/juanp/Downloads/archive\rating.csv


In [16]:
rating_path = "/Users/juanp/Downloads/archive/rating.csv"
anime_path = "/Users/juanp/Downloads/archive/anime.csv"

In [17]:
rating_df = pd.read_csv(rating_path)
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [19]:
anime_df = pd.read_csv(anime_path)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [20]:
print(f"anime set (row, col): {anime_df.shape}\n\nrating set (row , col): {rating_df.shape}")

anime set (row, col): (12294, 7)

rating set (row , col): (7813737, 3)


In [23]:
print("Anime: \n")
print(anime_df.info())
print("\n", "*"*50, "\nRating:\n")
print(rating_df.info())

Anime: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None

 ************************************************** 
Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


In [27]:
print("Anime missing values (%):\n")
print(round(anime_df.isnull().sum().sort_values(ascending=False)/len(anime_df.index),4)*100)
print("\n", "*"*50,"\n\Rating missing values (%):\n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df.index),4)*100)

Anime missing values (%):

rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64

 ************************************************** 
\Rating missing values (%):

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [31]:
print(anime_df['type'].mode())
print(anime_df['genre'].mode())

0    TV
dtype: object
0    Hentai
dtype: object


In [33]:
#Deleting anime with 0 rating
anime_df=anime_df[~np.isnan(anime_df["rating"])]

#Filling mode value for genre and type
anime_df['genre'] = anime_df['genre'].fillna(anime_df['genre'].dropna().mode().values[0])
anime_df['type'] = anime_df['type'].fillna(anime_df['type'].dropna().mode().values[0])

#Checking if all null values are filled
anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [34]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x == -1 else x)
rating_df.head(20)

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
5,1,355,NaN
6,1,356,NaN
7,1,442,NaN
8,1,487,NaN
9,1,846,NaN


In [36]:
# Recomment anime series. Type == TV
anime_df = anime_df[anime_df['type'] == 'TV']

# New Dataframe combining both anime and rating on the anime_id column.
rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user',''])

# Leaving only user_id, name and rating
rated_anime = rated_anime[['user_id', 'name', 'rating']]

# 0nly we compute our Df based only on the first 7500 users.(For computing purpose)
rated_anime_7500 = rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()

,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


In [37]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values = 'rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [38]:
# Value normalization
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis = 1)

# Filling Nan values as 0
pivot_n.fillna(0, inplace=True)

# Transposing the pivot for the next step.
pivot_n = pivot_n.T

# Dropping columns with the values of 0 (unrated).
pivot_n = pivot_n.loc[:, (pivot_n != 0 ).any(axis=0)]

# Using scipy package to convert to sparse matrix format for the similarity computation.
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [39]:
# Model based on anime similarity
anime_similarity = cosine_similarity(piv_sparse)

# DF of anime similarities
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [49]:
def anime_recommendation(ani_name):
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by=ani_name, ascending=False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number += 1
        """
This function will return the top 5 shows with the highest cosine similarity value and show match percent

example:
    >>>Input: 
        anime_recommendation('Death Note')
        
    >>>Output: 
        Recommended because you watched Death Note:

                    #1: Code Geass: Hangyaku no Lelouch, 57.35% match
                    #2: Code Geass: Hangyaku no Lelouch R2, 54.81% match
                    #3: Fullmetal Alchemist, 51.07% match
                    #4: Shingeki no Kyojin, 48.68% match
                    #5: Fullmetal Alchemist: Brotherhood, 45.99% match              
    """

    

In [68]:
anime_recommendation("Akame ga Kill!")

Recommended because you watched Akame ga Kill!:

#1: Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka, 49.88% match
#2: Mahouka Koukou no Rettousei, 42.5% match
#3: Charlotte, 40.94% match
#4: Rakudai Kishi no Cavalry, 40.7% match
#5: Nisekoi, 40.26% match
